In [1]:
# This progrtma dects if an email is spam(1) or not spam (0)

In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [4]:
# load the data
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [5]:
# data from kaggle : https://www.kaggle.com/datasets/karthickveerakumar/spam-filter

In [8]:
#read the file
df = pd.read_csv("emails.csv")

In [14]:
# start with EDA

In [10]:
df.head(6)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1


In [11]:
# get the number or rows and columns in our dataset
df.shape

(5728, 2)

In [12]:
# get the columns names
df.columns

Index(['text', 'spam'], dtype='object')

In [13]:
# check for duplicates
df.drop_duplicates(inplace=True)
#show the new shape afer drop duplicates
df.shape

(5695, 2)

In [15]:
# lets see if we have missing data in each column (NAN,NaN,na)
df.isnull().sum()

text    0
spam    0
dtype: int64

In [16]:
#download stopwords packages
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
# stopwords in natural language processing are useless words or data
# create a function used to process the text
# remove punctuation from text
# retun a list a list of clean data

def process_text(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = "".join(nopunc)

  # remove stopwords
  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words("english")]

  return clean_words

In [24]:
# lets start with tokenization (a list of tokens also called lemmas)
# tokens : each word is seperated by a comma
df["text"].head(5).apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [29]:
# quick explanation:
message3 = "hello world hello world hello word play " # (imagine this is a message )
message6 = "test test test test test "
print(message3)
print()

# convert text to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
bag_of_words = CountVectorizer(analyzer=process_text).fit_transform([[message3], [message6]] )
print(bag_of_words)
print(bag_of_words.shape)

hello world hello world hello word play 

  (0, 0)	3
  (0, 4)	2
  (0, 3)	1
  (0, 1)	1
  (1, 2)	5
(2, 5)


In [30]:
# the message 3,  is represented by 0 with 3 unique words, 2 words that are same world, 1 world that is the same is play
# message 6, the 1 represents 1 unique word , and that word repeats 5 times
# we will transform our data into token counts

In [31]:
# convert a collection of text to a matrix of tokens
messages_bag_of_words = CountVectorizer(analyzer=process_text).fit_transform(df["text"] )

In [33]:
# split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bag_of_words, df["spam"], test_size=0.20, random_state = 0)
#  X_train = our feature training data set
# X_test = feature testing set
# y_train = target traing set
# y_test = target testing set


In [34]:
# get shape of data
messages_bag_of_words.shape

(5695, 37229)

In [37]:
# Create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)

In [38]:
# print the preditions
print(classifier.predict(X_train))

[0 0 0 ... 0 0 0]


In [39]:
# print the actual target values
print(y_train.values)

[0 0 0 ... 0 0 0]


In [44]:
# cant really see the values, hard to tell if the align
# lets use other methods to evaluate the modelperformance on the trainig datset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(y_train, pred))
print()
print("Accuracy: ", accuracy_score(y_train, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556


Confusion Matrix: 
 [[3445   12]
 [   1 1098]]

Accuracy:  0.9971466198419666


In [45]:
# Lets see performance on test data
print(y_test.values)
print(classifier.predict(X_test))

[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]


In [46]:
pred = classifier.predict(X_test)
print(classification_report(y_test, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(y_test, pred))
print()
print("Accuracy: ", accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139


Confusion Matrix: 
 [[862   8]
 [  1 268]]

Accuracy:  0.9920983318700615


In [47]:
# the accuracy droped a little but still 99% (100% = 1)
# this is a great model